#### **Mlflow tracking server**

In [1]:
import mlflow

# setting tracking uri.
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# setting an localhost connection.
mlflow.set_experiment("Check localhost connection")

with mlflow.start_run():
    mlflow.log_metric("test", 1)
    mlflow.log_metric("netra",2)

/Users/netrakc/Desktop/kn-mlops/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


KeyboardInterrupt: 

In [2]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import mlflow
from mlflow.models import infer_signature

In [3]:
# set the tracking uri. 
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Load the dataset. 
X,y = datasets.load_iris(return_X_y=True)

# split the data into train and test sets. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# Define the model hyperparameters. 
params = {
    "penalty": "l2",
    "solver": "lbfgs",
    "max_iter": 1000, 
    "multi_class": "auto",
    "random_state": 8888
}

lr = LogisticRegression(**params)

lr.fit(X_train, y_train)

/Users/netrakc/Desktop/kn-mlops/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [4]:
X_test

array([[4.8, 3. , 1.4, 0.3],
       [6.4, 3.1, 5.5, 1.8],
       [5.1, 2.5, 3. , 1.1],
       [6.1, 2.9, 4.7, 1.4],
       [6.7, 3.3, 5.7, 2.5],
       [5. , 3.4, 1.5, 0.2],
       [7.2, 3. , 5.8, 1.6],
       [7.2, 3.2, 6. , 1.8],
       [5. , 3.5, 1.6, 0.6],
       [5.9, 3. , 4.2, 1.5],
       [5.1, 3.7, 1.5, 0.4],
       [4.7, 3.2, 1.3, 0.2],
       [6.2, 2.8, 4.8, 1.8],
       [6.4, 2.8, 5.6, 2.2],
       [7.7, 3. , 6.1, 2.3],
       [4.6, 3.1, 1.5, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [6.5, 3. , 5.8, 2.2],
       [5.9, 3.2, 4.8, 1.8],
       [5.2, 4.1, 1.5, 0.1],
       [6. , 3. , 4.8, 1.8],
       [5.2, 3.5, 1.5, 0.2],
       [5.4, 3.4, 1.7, 0.2],
       [6.3, 2.9, 5.6, 1.8],
       [5. , 2.3, 3.3, 1. ],
       [6.1, 3. , 4.9, 1.8],
       [5.5, 2.5, 4. , 1.3],
       [5.7, 2.8, 4.5, 1.3],
       [7.7, 3.8, 6.7, 2.2]])

In [5]:
# prediction on the test set. 
y_pred = lr.predict(X_test)
y_pred

array([0, 2, 1, 1, 2, 0, 2, 2, 0, 1, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2,
       0, 0, 2, 1, 2, 1, 1, 2])

In [6]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9666666666666667


In [8]:
# mlflow tracking.
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# create a new mlflow experiment.
mlflow.set_experiment("Mlflow Quickstart")

# start an mlflow run. 
with mlflow.start_run():
    # Log the hyperparameters. 
    mlflow.log_params(params)
    
    # Log the accuracy metrics. 
    mlflow.log_metric("accuracy", accuracy)

    # Setting an tags for an experiment. 
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature. 
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model. 
    model_info = mlflow.sklearn.log_model(
        sk_model = lr, 
        artifact_path = "iris_model",
        signature = signature, 
        input_example = X_train, 
        registered_model_name="tracking-quickstart" # this is like, I am saying this is the best model, registered in mlflow. 
    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/06/11 10:23:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2


🏃 View run wise-roo-319 at: http://127.0.0.1:5000/#/experiments/720752019416898048/runs/724ab9d6edc14b5bb013276aa044da66
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/720752019416898048


Created version '2' of model 'tracking-quickstart'.


In [9]:
model_info.model_uri

'runs:/724ab9d6edc14b5bb013276aa044da66/iris_model'

#### **Inferencing and Validating Model**

In [11]:
import mlflow
logged_model = 'runs:/724ab9d6edc14b5bb013276aa044da66/iris_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(data))

NameError: name 'data' is not defined

In [12]:
import mlflow
from pyspark.sql.functions import struct, col
logged_model = 'runs:/724ab9d6edc14b5bb013276aa044da66/iris_model'

# Load model as a Spark UDF. Override result_type if the model does not return double values.
loaded_model = mlflow.pyfunc.spark_udf(spark, model_uri=logged_model)

# Predict on a Spark DataFrame.
df.withColumn('predictions', loaded_model(struct(*map(col, df.columns))))

ModuleNotFoundError: No module named 'pyspark'

#### **Load the model back for prediction as a generic python function model.**

In [13]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_features_name = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_features_name)
result["actual_class"] = y_test
result["predicted_class"] = predictions

results

MlflowException: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/720752019416898048/724ab9d6edc14b5bb013276aa044da66/artifacts:
##### File iris_model/requirements.txt #####
API request to http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/720752019416898048/724ab9d6edc14b5bb013276aa044da66/artifacts/iris_model/requirements.txt failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/720752019416898048/724ab9d6edc14b5bb013276aa044da66/artifacts/iris_model/requirements.txt (Caused by ResponseError('too many 500 error responses'))
##### File iris_model/python_env.yaml #####
API request to http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/720752019416898048/724ab9d6edc14b5bb013276aa044da66/artifacts/iris_model/python_env.yaml failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries excee

#### **Model Registry**

#### **Inferencing from model from model registry.**

In [14]:
import mlflow.sklearn
model_name = "tracking-quickstart"
model_version = "latest"

model_uri = f"models:/{model_name}/{model_version}"

mlflow.sklearn.load_model(model_uri)
model

KeyboardInterrupt: 

In [ ]:
model_uri

In [ ]:
y_pred_new = model.predict(X_test)
y_pred_new